In [1]:
import csv
import pandas as pd
from geopy.distance import geodesic

# 读取地铁站数据
tube_stations = []
with open('tube.csv', 'r') as tube_file:
    tube_reader = csv.DictReader(tube_file)
    for row in tube_reader:
        station_lat = float(row['y'])  # 假设经纬度在'y'列
        station_lng = float(row['x'])  # 假设经纬度在'x'列
        station_name = row['NAME']  # 假设地铁站名在'name'列
        tube_stations.append({'name': station_name, 'lat': station_lat, 'lng': station_lng})

# 读取厕所数据
public_toilets = []
with open('toilets_accessible.csv', 'r') as toilet_file:
    toilet_reader = csv.DictReader(toilet_file)
    for row in toilet_reader:
        toilet_lat = float(row['lat'])  # 假设经纬度在'lat'列
        toilet_lng = float(row['lng'])  # 假设经纬度在'lng'列
        public_toilets.append({'lat': toilet_lat, 'lng': toilet_lng})

k = 1  # 系数
beta = 1  # 距离的指数
Mj = 1  # 厕所的服务能力，设为相等

# 创建一个空的 DataFrame 来存储地铁站的可达性
tube_accessibility = pd.DataFrame(columns=['Station', 'Accessibility'])

# 遍历每个地铁站
for station in tube_stations:
    station_name = station['name']
    station_lat = station['lat']
    station_lng = station['lng']
    station_accessibility = 0
    
    # 遍历每个厕所，计算可达性并累加
    for toilet in public_toilets:
        toilet_lat = toilet['lat']
        toilet_lng = toilet['lng']
        distance = geodesic((station_lat, station_lng), (toilet_lat, toilet_lng)).kilometers
        accessibility = k * (Mj / distance**beta)
        
        # 将可达性累加到地铁站的可达性中
        station_accessibility += accessibility
    
    # 将地铁站的可达性添加到 DataFrame 中
    tube_accessibility = tube_accessibility.append({'Station': station_name, 'Accessibility': station_accessibility}, ignore_index=True)

    # 设置 pandas 显示的最大行数和最大列数
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# 打印地铁站的可达性表格
print(tube_accessibility)

# 读取已有的 tube.csv 文件
existing_tube_data = pd.read_csv('tube.csv')

# 合并数据
merged_data = pd.merge(existing_tube_data, tube_accessibility, left_on='NAME', right_on='Station', how='left')

# 保存合并后的数据到新的 tube.csv 文件
merged_data.to_csv('access_tube1.csv', index=False)



<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)
/tmp/ipykernel_79943/476444821.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility = tube_accessibility.append({'Station': station_name, 'Accessibility': station_accessibility}, ignore_index=True)
/tmp/ipykernel_79943/476444821.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility = tube_accessibility.append({'Station': station_name, 'Accessibility': station_accessibility}, ignore_index=True)
/tmp/ipykernel_79943/476444821.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility = tube_accessibility.append({'Station': station_nam

                     Station  Accessibility
0                    Aldgate     140.167857
1               Aldgate East     119.118792
2                      Angel     100.571564
3               Baker Street      99.014861
4                       Bank     152.082201
5                   Barbican     126.622459
6                  Bayswater      84.928150
7                Blackfriars     126.411857
8                Bond Street     109.409342
9                    Borough     106.756906
10             Cannon Street     166.176885
11             Chancery Lane     126.098410
12             Charing Cross     134.655111
13             Covent Garden     151.494338
14              Earl's Court      82.722751
15   Edgware Road (Bakerloo)      91.044739
16                Embankment     139.431970
17                    Euston     120.025198
18             Euston Square     123.949516
19                Farringdon     124.383325
20           Gloucester Road     111.951303
21             Goodge Street    

/tmp/ipykernel_79943/476444821.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility = tube_accessibility.append({'Station': station_name, 'Accessibility': station_accessibility}, ignore_index=True)


In [2]:
# 按可达性降序排序
tube_accessibility = tube_accessibility.sort_values(by='Accessibility', ascending=False)

# 打印地铁站的可达性表格
print(tube_accessibility)


                     Station  Accessibility
10             Cannon Street     166.176885
4                       Bank     152.082201
13             Covent Garden     151.494338
37                  Monument     151.370094
53                St. Paul's     149.305291
34             Mansion House     146.402032
0                    Aldgate     140.167857
31          Leicester Square     139.490161
16                Embankment     139.431970
12             Charing Cross     134.655111
32          Liverpool Street     134.008767
43         Piccadilly Circus     131.856255
55      Tottenham Court Road     131.177035
38                  Moorgate     129.874323
5                   Barbican     126.622459
7                Blackfriars     126.411857
11             Chancery Lane     126.098410
54                    Temple     125.602542
59             Warren Street     125.345866
19                Farringdon     124.383325
18             Euston Square     123.949516
25                   Holborn    

In [3]:
# 计算最大值、最小值、平均值和标准差
max_accessibility = tube_accessibility['Accessibility'].max()
min_accessibility = tube_accessibility['Accessibility'].min()
mean_accessibility = tube_accessibility['Accessibility'].mean()
std_accessibility = tube_accessibility['Accessibility'].std()

# 打印计算结果
print(f"最大可达性值: {max_accessibility}")
print(f"最小可达性值: {min_accessibility}")
print(f"平均可达性值: {mean_accessibility}")
print(f"标准差: {std_accessibility}")

最大可达性值: 166.17688454565896
最小可达性值: 82.72275075284139
平均可达性值: 116.10430176127396
标准差: 19.173431987283273


In [4]:
beta15 = 1.5  # 距离的指数


# 创建一个空的 DataFrame 来存储地铁站的可达性
tube_accessibility15 = pd.DataFrame(columns=['Station', 'Accessibility1.5'])

# 遍历每个地铁站
for station in tube_stations:
    station_name = station['name']
    station_lat = station['lat']
    station_lng = station['lng']
    station_accessibility15 = 0
    
    # 遍历每个厕所，计算可达性并累加
    for toilet in public_toilets:
        toilet_lat = toilet['lat']
        toilet_lng = toilet['lng']
        distance = geodesic((station_lat, station_lng), (toilet_lat, toilet_lng)).kilometers
        accessibility15 = k * (Mj / distance**beta15)
        
        # 将可达性累加到地铁站的可达性中
        station_accessibility15 += accessibility15
        # 在原有的tube.csv文件中新增一个Accessibility列，并将可达性值写入对应行

            
    # 将地铁站的可达性添加到 DataFrame 中
    tube_accessibility15 = tube_accessibility15.append({'Station': station_name, 'Accessibility1.5': station_accessibility15}, ignore_index=True)
# 按可达性降序排序
tube_accessibility15 = tube_accessibility15.sort_values(by='Accessibility1.5', ascending=False)

# 打印地铁站的可达性表格
print(tube_accessibility15)
# 计算最大值、最小值、平均值和标准差
max_accessibility15 = tube_accessibility15['Accessibility1.5'].max()
min_accessibility15 = tube_accessibility15['Accessibility1.5'].min()
mean_accessibility15 = tube_accessibility15['Accessibility1.5'].mean()
std_accessibility15 = tube_accessibility15['Accessibility1.5'].std()

# 打印计算结果
print(f"最大可达性值: {max_accessibility15}")
print(f"最小可达性值: {min_accessibility15}")
print(f"平均可达性值: {mean_accessibility15}")
print(f"标准差: {std_accessibility15}")

# 读取已有的 tube.csv 文件
existing_tube_data = pd.read_csv('tube.csv')

# 合并数据
merged_data = pd.merge(existing_tube_data, tube_accessibility15, left_on='NAME', right_on='Station', how='left')

# 保存合并后的数据到新的 tube.csv 文件
merged_data.to_csv('access_tube15.csv', index=False)


/tmp/ipykernel_79943/406840153.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility15 = tube_accessibility15.append({'Station': station_name, 'Accessibility1.5': station_accessibility15}, ignore_index=True)
/tmp/ipykernel_79943/406840153.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility15 = tube_accessibility15.append({'Station': station_name, 'Accessibility1.5': station_accessibility15}, ignore_index=True)
/tmp/ipykernel_79943/406840153.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility15 = tube_accessibility15.append({'Station': station_name, 'Accessibility1.5': station_accessibility15}, ignore_index=True)
/tmp/ipykernel_79943/406840153.py:27: FutureWarning: Th

                     Station  Accessibility1.5
10             Cannon Street        222.837298
13             Covent Garden        200.087397
20           Gloucester Road        181.477517
0                    Aldgate        181.012284
37                  Monument        179.752752
4                       Bank        158.788936
53                St. Paul's        157.215599
34             Mansion House        140.388286
16                Embankment        135.788555
31          Leicester Square        127.154896
59             Warren Street        125.205611
55      Tottenham Court Road        120.987115
32          Liverpool Street        117.453390
12             Charing Cross        117.260394
18             Euston Square        115.456825
43         Piccadilly Circus        114.209254
29             Lambeth North        102.635717
38                  Moorgate        101.761721
17                    Euston        100.810002
58                  Victoria         99.975870
1            

/tmp/ipykernel_79943/406840153.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility15 = tube_accessibility15.append({'Station': station_name, 'Accessibility1.5': station_accessibility15}, ignore_index=True)


In [5]:
beta18 = 1.8  # 距离的指数


# 创建一个空的 DataFrame 来存储地铁站的可达性
tube_accessibility18 = pd.DataFrame(columns=['Station', 'Accessibility1.8'])

# 遍历每个地铁站
for station in tube_stations:
    station_name = station['name']
    station_lat = station['lat']
    station_lng = station['lng']
    station_accessibility18 = 0
    
    # 遍历每个厕所，计算可达性并累加
    for toilet in public_toilets:
        toilet_lat = toilet['lat']
        toilet_lng = toilet['lng']
        distance = geodesic((station_lat, station_lng), (toilet_lat, toilet_lng)).kilometers
        accessibility18 = k * (Mj / distance**beta18)
        
        # 将可达性累加到地铁站的可达性中
        station_accessibility18 += accessibility18
    

            
    # 将地铁站的可达性添加到 DataFrame 中
    tube_accessibility18 = tube_accessibility18.append({'Station': station_name, 'Accessibility1.8': station_accessibility18}, ignore_index=True)
# 按可达性降序排序
tube_accessibility18 = tube_accessibility18.sort_values(by='Accessibility1.8', ascending=False)

# 打印地铁站的可达性表格
print(tube_accessibility18)
# 计算最大值、最小值、平均值和标准差
max_accessibility18 = tube_accessibility18['Accessibility1.8'].max()
min_accessibility18 = tube_accessibility18['Accessibility1.8'].min()
mean_accessibility18 = tube_accessibility18['Accessibility1.8'].mean()
std_accessibility18 = tube_accessibility18['Accessibility1.8'].std()

# 打印计算结果
print(f"最大可达性值: {max_accessibility18}")
print(f"最小可达性值: {min_accessibility18}")
print(f"平均可达性值: {mean_accessibility18}")
print(f"标准差: {std_accessibility18}")

# 读取已有的 tube.csv 文件
existing_tube_data = pd.read_csv('tube.csv')

# 合并数据
merged_data = pd.merge(existing_tube_data, tube_accessibility18, left_on='NAME', right_on='Station', how='left')

# 保存合并后的数据到新的 tube.csv 文件
merged_data.to_csv('access_tube18.csv', index=False)

/tmp/ipykernel_79943/586883853.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility18 = tube_accessibility18.append({'Station': station_name, 'Accessibility1.8': station_accessibility18}, ignore_index=True)
/tmp/ipykernel_79943/586883853.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility18 = tube_accessibility18.append({'Station': station_name, 'Accessibility1.8': station_accessibility18}, ignore_index=True)
/tmp/ipykernel_79943/586883853.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility18 = tube_accessibility18.append({'Station': station_name, 'Accessibility1.8': station_accessibility18}, ignore_index=True)
/tmp/ipykernel_79943/586883853.py:27: FutureWarning: Th

                     Station  Accessibility1.8
20           Gloucester Road        406.722697
10             Cannon Street        350.847490
13             Covent Garden        346.529428
0                    Aldgate        315.128402
37                  Monument        270.085359
53                St. Paul's        210.035422
4                       Bank        201.101374
59             Warren Street        177.221429
16                Embankment        174.088995
34             Mansion House        166.832916
55      Tottenham Court Road        155.760599
18             Euston Square        153.201303
31          Leicester Square        147.918400
29             Lambeth North        134.838311
32          Liverpool Street        132.927321
12             Charing Cross        132.851080
43         Piccadilly Circus        131.839601
58                  Victoria        129.867600
17                    Euston        120.396089
61               Westminster        112.394256
1            

/tmp/ipykernel_79943/586883853.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility18 = tube_accessibility18.append({'Station': station_name, 'Accessibility1.8': station_accessibility18}, ignore_index=True)


In [6]:
beta2 = 2  # 距离的指数


# 创建一个空的 DataFrame 来存储地铁站的可达性
tube_accessibility2 = pd.DataFrame(columns=['Station', 'Accessibility2'])

# 遍历每个地铁站
for station in tube_stations:
    station_name = station['name']
    station_lat = station['lat']
    station_lng = station['lng']
    station_accessibility2 = 0
    
    # 遍历每个厕所，计算可达性并累加
    for toilet in public_toilets:
        toilet_lat = toilet['lat']
        toilet_lng = toilet['lng']
        distance = geodesic((station_lat, station_lng), (toilet_lat, toilet_lng)).kilometers
        accessibility2 = k * (Mj / distance**beta2)
        
        # 将可达性累加到地铁站的可达性中
        station_accessibility2 += accessibility2
    

    # 将地铁站的可达性添加到 DataFrame 中
    tube_accessibility2 = tube_accessibility2.append({'Station': station_name, 'Accessibility2': station_accessibility2}, ignore_index=True)
# 按可达性降序排序
tube_accessibility2 = tube_accessibility2.sort_values(by='Accessibility2', ascending=False)

# 打印地铁站的可达性表格
print(tube_accessibility2)
# 计算最大值、最小值、平均值和标准差
max_accessibility2 = tube_accessibility2['Accessibility2'].max()
min_accessibility2 = tube_accessibility2['Accessibility2'].min()
mean_accessibility2 = tube_accessibility2['Accessibility2'].mean()
std_accessibility2 = tube_accessibility2['Accessibility2'].std()

# 打印计算结果
print(f"最大可达性值: {max_accessibility2}")
print(f"最小可达性值: {min_accessibility2}")
print(f"平均可达性值: {mean_accessibility2}")
print(f"标准差: {std_accessibility2}")

# 读取已有的 tube.csv 文件
existing_tube_data = pd.read_csv('tube.csv')

# 合并数据
merged_data = pd.merge(existing_tube_data, tube_accessibility2, left_on='NAME', right_on='Station', how='left')

# 保存合并后的数据到新的 tube.csv 文件
merged_data.to_csv('access_tube2.csv', index=False)

/tmp/ipykernel_79943/1792075635.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility2 = tube_accessibility2.append({'Station': station_name, 'Accessibility2': station_accessibility2}, ignore_index=True)
/tmp/ipykernel_79943/1792075635.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility2 = tube_accessibility2.append({'Station': station_name, 'Accessibility2': station_accessibility2}, ignore_index=True)
/tmp/ipykernel_79943/1792075635.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility2 = tube_accessibility2.append({'Station': station_name, 'Accessibility2': station_accessibility2}, ignore_index=True)
/tmp/ipykernel_79943/1792075635.py:26: FutureWarning: The frame.app

                     Station  Accessibility2
20           Gloucester Road      753.542709
13             Covent Garden      552.640664
10             Cannon Street      509.058228
0                    Aldgate      505.950161
37                  Monument      386.710409
53                St. Paul's      276.146179
4                       Bank      249.158465
59             Warren Street      246.982618
16                Embankment      223.203408
18             Euston Square      205.195104
55      Tottenham Court Road      204.289559
34             Mansion House      197.905042
29             Lambeth North      182.820674
31          Leicester Square      173.828657
58                  Victoria      171.716621
43         Piccadilly Circus      156.564387
12             Charing Cross      153.808436
32          Liverpool Street      153.700431
17                    Euston      149.429866
61               Westminster      144.683800
1               Aldgate East      132.325862
41        

/tmp/ipykernel_79943/1792075635.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility2 = tube_accessibility2.append({'Station': station_name, 'Accessibility2': station_accessibility2}, ignore_index=True)
